In [2]:
from sklearn import svm
import imp
import pandas as pd
import nltk
import re
from sklearn import svm
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,classification_report
import keras
from keras.models import Sequential
from keras.layers import Dense

C:\Users\nel_1\AppData\Local\Temp\ipykernel_10892\3433365118.py:2: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  import imp


In [3]:
true_df = pd.read_csv('./True.csv')
fake_df = pd.read_csv('./Fake.csv')
true_df['label'] = 0
fake_df['label'] = 1
true_df = true_df[['text','label']]
fake_df = fake_df[['text','label']]
dataset = pd.concat([true_df , fake_df])
print(dataset['label'].value_counts())

1    23481
0    21417
Name: label, dtype: int64


After reading our data in to dataframes, it's time to clean it.</br>
to cleanse this data a function is defined to convert the upper case to lower case, substitute non English words with space and tokenize it.

In [4]:
dataset = dataset.sample(frac = 1)
lemmatizer = WordNetLemmatizer()
stopwords = set(stopwords.words('english'))
def clean_data(text):
    text = text.lower() 
    text = re.sub('[^a-zA-Z]' , ' ' , text)
    token = text.split() 
    news = [lemmatizer.lemmatize(word) for word in token if not word in stopwords]  
    clean_news = ' '.join(news) 
    
    return clean_news 

</br>
</br>

to save runtime, we only read 20000 rows of our dataframes. 

In [5]:
dataset['text'] = dataset['text'].apply(lambda x : clean_data(x))
vectorizer = TfidfVectorizer(max_features = 50000 , lowercase=False , ngram_range=(1,2))
X = dataset.iloc[:10000,0]
y = dataset.iloc[:10000,1]
print(X.head())
print(y.head())

4995     reuters u president donald trump nominee head ...
2792     donald trump got as handed gold star father si...
6790     itching taste life america president donald tr...
6692     woman make percent country grossly underrepres...
10934    knew tiffany co decided show hand come politic...
Name: text, dtype: object
4995     0
2792     1
6790     1
6692     1
10934    1
Name: label, dtype: int64


next step is to split our data into train and test and use vectorizer on them to map words or phrases from vocabulary to a corresponding vector of real numbers.

In [6]:
train_X , test_X , train_y , test_y = train_test_split(X , y , test_size = 0.2 ,random_state = 0)
vec_train = vectorizer.fit_transform(train_X)
vec_train = vec_train.toarray()
vec_test = vectorizer.transform(test_X).toarray()
train_data = pd.DataFrame(vec_train , columns=vectorizer.get_feature_names_out())
test_data = pd.DataFrame(vec_test , columns= vectorizer.get_feature_names_out())

### implementing the model and the result:

In [7]:
model = Sequential()
model.add(Dense(units = 100 , activation = 'relu' , input_dim = vec_train.shape[1]))
model.add(Dense(units = 50 , activation = 'relu'))
model.add(Dense(units = 25 , activation = 'relu'))
model.add(Dense(units = 10 , activation = 'relu'))
model.add(Dense(units = 1 , activation = 'sigmoid'))
model.compile(optimizer = 'adam' , loss = 'binary_crossentropy' , metrics = ['accuracy'])
model.fit(vec_train,train_y , epochs = 5)

Epoch 1/5
250/250 [==============================] - 12s 45ms/step - loss: 0.1724 - accuracy: 0.9589
Epoch 2/5
250/250 [==============================] - 11s 44ms/step - loss: 0.0049 - accuracy: 0.9992
Epoch 3/5
250/250 [==============================] - 12s 47ms/step - loss: 0.0014 - accuracy: 0.9998
Epoch 4/5
250/250 [==============================] - 12s 48ms/step - loss: 0.0016 - accuracy: 0.9999
Epoch 5/5
250/250 [==============================] - 12s 47ms/step - loss: 6.5390e-04 - accuracy: 0.9999
